In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import (roc_auc_score, average_precision_score, precision_score, recall_score, classification_report, f1_score)
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import (roc_auc_score, average_precision_score, precision_score, recall_score, classification_report, f1_score)
import matplotlib.pyplot as plt

from sklearn.inspection import permutation_importance
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [40]:
!pip install dagshub
!pip install mlflow
import dagshub
import mlflow

dagshub.init(repo_owner='kechik21',repo_name='ML_HW2',mlflow=True)

Initialized MLflow to track repo "kechik21/ML_HW2"

Repository kechik21/ML_HW2 initialized!

In [41]:
# Training data
train_tr = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
train_id = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')

train = train_tr.merge(train_id, on='TransactionID', how='left')
print(train_tr.shape)
print(train_id.shape)

(590540, 394)
(144233, 41)


In [42]:
print("Train Data Columns:")
print(train.columns)

Train Data Columns:
Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=434)


In [43]:
X = train.drop(['TransactionID', 'isFraud'], axis=1)
y = train['isFraud']

In [44]:
# Differentiate categorical and numerical
cat_coluka = ['userId','P_emaildomain','R_emaildomain','DeviceType','DeviceInfo','ProductCD','addr1', 'addr2',]
cat_cards = ['card' + str(i) for i in range(1, 7)]
cat_ms = ['M' + str(i) for i in range(1, 10)]
cat_ids = ['id_' + str(i) for i in range(12, 39)]
cat_cols = cat_coluka + cat_cards + cat_ms + cat_ids

print("Categorical Columns:")
print(cat_cols)
print("\nCard Columns:")
print(cat_cards)
print("\nM Columns:")
print(cat_ms)
print("\nID Columns:")
print(cat_ids)
print("\nAll Categorical Features:")
print(cat_cols)
print("\nCategorical Columns Amount:")
print(len(cat_cols))


num_cols = [col for col in X.columns if col not in cat_cols and col != 'isFraud']

print("\nNumerical Columns Amount:")
print(len(num_cols))

Categorical Columns:
['userId', 'P_emaildomain', 'R_emaildomain', 'DeviceType', 'DeviceInfo', 'ProductCD', 'addr1', 'addr2', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']

Card Columns:
['card1', 'card2', 'card3', 'card4', 'card5', 'card6']

M Columns:
['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']

ID Columns:
['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']

All Categorical Features:
['userId', 'P_emaildomain', 'R_emaildomain', 'DeviceType', 'DeviceInfo', 'ProductCD', 'addr1', 'addr2',

Filling missing values for numeric and categorical

# Data Cleaning

In [45]:
class FillNa(BaseEstimator, TransformerMixin):
    def __init__(self, num_cols, cat_cols):
        self.num_cols = num_cols
        self.cat_cols = cat_cols
        self.values = {}

    def fit(self, X, y=None):
        for col in self.num_cols:
            if col in X.columns:
                val = X[col].median()
                X[col] = X[col].fillna(val)
                self.values[col] = val

        for c in self.cat_cols:
            if c in X.columns:  
                if X[c].dtype in ['int64', 'float64']:
                    X[c] = X[c].astype(str)
                X[c] = X[c].fillna('NotAv')
        return self

    def transform(self, X):
        for key, value in self.values.items():
            if key in X.columns:
                X[key] = X[key].fillna(value)

        for c in self.cat_cols:
            if c in X.columns:  
                if X[c].dtype in ['int64', 'float64']:
                    X[c] = X[c].astype(str)
                X[c] = X[c].fillna('NotAv')

        return X

In [46]:
class MissingValueDropper(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=90):
        self.columns_to_drop = None
        self.threshold = threshold

    def fit(self, X, y=None):
        missing_cols = [col for col in X.columns if X[col].isna().any()]
        high_missing_cols = [
            col for col in missing_cols 
            if (X[col].isna().mean() * 100) > self.threshold
        ]
        
        self.columns_to_drop = high_missing_cols
        print(f'Identified {len(high_missing_cols)} columns to drop')
        return self

    def transform(self, X):
        if self.columns_to_drop is None:
            raise ValueError('Needs Fitting !')
        
        return X.drop(columns=self.columns_to_drop)

# Feature Engineering

In [47]:
from sklearn.base import BaseEstimator, TransformerMixin
from category_encoders import WOEEncoder
import pandas as pd

class CatEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=2):
        self.threshold = threshold
        self.binary_cols = []
        self.multi_cols = []
        self.encoder = None

    def fit(self, X, y):
        X = X.copy()
        
        for col in X.columns:
            if X[col].dtype == 'object':
                non_rep = X[col].nunique()
                if non_rep <= self.threshold:
                    self.binary_cols.append(col)
                else:
                    self.multi_cols.append(col)

        self.encoder = WOEEncoder(cols=self.multi_cols)
        self.encoder.fit(X[self.multi_cols], y)
        
        return self

    def transform(self, X):
        X = X.copy()
        if self.multi_cols:
            X_woe = self.encoder.transform(X[self.multi_cols])
            X[self.multi_cols] = X_woe

        if self.binary_cols:
            X = pd.get_dummies(X, columns=self.binary_cols, drop_first=True, dtype=int)
        
        return X

In [48]:
class HighCorrelationDropper(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.85):
        self.threshold = threshold
        self.high_corr_cols_ = None  
        
    def fit(self, X, y=None):
       
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame")
        corr_matrix = X.corr().abs()
        upper_triangle = np.triu(corr_matrix, k=1)
        highly_correlated = set()
        for col in corr_matrix.columns:
            correlated_with = corr_matrix.index[upper_triangle[:, corr_matrix.columns.get_loc(col)] > self.threshold]
            highly_correlated.update(correlated_with)
            
        self.high_corr_cols_ = list(highly_correlated)
        print(f"Identified {len(self.high_corr_cols_)} highly correlated features to remove")
        
        return self
        
    def transform(self, X):
        if self.high_corr_cols_ is None:
            raise RuntimeError("Need Fitting ! ")
            
        return X.drop(columns=self.high_corr_cols_)

# Feature Selection

In [49]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor 
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class RFFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        target_feature_count=150,
        rf_trees=100,
        max_depth=6,
        min_samples_split=5,
        min_samples_leaf=4,
        max_features='sqrt',
        parallel_jobs=-1,
        seed=42,
        verbose=0
    ):
      
        self.target_feature_count = target_feature_count
        self.rf_trees = rf_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.parallel_jobs = parallel_jobs
        self.seed = seed
        self.verbose = verbose
        self.kept_features_ = None
        self.feature_mask_ = None
        self.feature_ranks_ = None

    def fit(self, X, y):
        print("Starting feature selection...")
        
        rf_model = RandomForestRegressor(
            n_estimators=self.rf_trees,
            max_depth=self.max_depth,
            min_samples_split=self.min_samples_split,
            min_samples_leaf=self.min_samples_leaf,
            max_features=self.max_features,
            n_jobs=self.parallel_jobs,
            random_state=self.seed,
            verbose=self.verbose
        )
        
        if X.shape[1] > 200:
            print(" Phase 1: Fast elimination of weak features...")
            rough_selector = RFE(
                estimator=rf_model,
                n_features_to_select=min(200, X.shape[1]),
                step=max(1, X.shape[1] // 20)  
            )
            rough_selector.fit(X, y)
            X = X.loc[:, rough_selector.support_]
        
        print(" Phase 2: Fine-tuning feature set...")
        final_selector = RFE(
            estimator=rf_model,
            n_features_to_select=self.target_feature_count,
            step=1 
        )
        final_selector.fit(X, y)
        
        if hasattr(X, 'columns'):
            self.kept_features_ = X.columns[final_selector.support_].tolist()
        self.feature_mask_ = final_selector.support_
        self.feature_ranks_ = final_selector.ranking_
        
        print(f"Done! Selected {len(self.kept_features_)} features.")
        return self

    def transform(self, X):
       
        if self.kept_features_ is None:
            raise RuntimeError("Fit the selector first!")
        
        if isinstance(X, pd.DataFrame):
            return X[self.kept_features_]
        return X[:, self.feature_mask_]

    def get_feature_indices(self, as_indices=False):
       
        return np.where(self.feature_mask_)[0] if as_indices else self.feature_mask_

In [50]:
print('The Classes I created are working so far')

The Classes I created are working so far


# Training

In [51]:
kva = train.drop(['TransactionID', 'isFraud'], axis=1)
target = train['isFraud']

X_train, X_val, y_train, y_val = train_test_split(kva,target,test_size=0.3,stratify=target,random_state=42)

In [52]:
mlflow.set_tracking_uri('https://dagshub.com/kechik21/ML_HW2.mlflow')
mlflow.set_experiment("Fraud_Detection_RandomForest")

class_weights = {
    0: 1.0,
    1: (len(y_train) - sum(y_train)) / sum(y_train)
}

fraud_pipeline = Pipeline([
    # Data Cleaning
    ('missing_value_handler', MissingValueDropper(threshold=90)),
    ('data_imputer', FillNa(num_cols=num_cols, cat_cols=cat_cols)),
    
    # Feature Engineering
    ('categorical_processor', CatEncoder(threshold=2)),
    ('correlation_filter', HighCorrelationDropper(threshold=0.85)),
    ('feature_selector', RFFeatureSelector(
        target_feature_count=int(len(X_train.columns)*0.8),
        rf_trees=100,
        max_depth=10,
        min_samples_split=5
    )),
    
     ('fraud_classifier', RandomForestClassifier(
        n_estimators=200,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=4,
        max_features='sqrt',
        class_weight=class_weights,
        n_jobs=-1,
        random_state=42,
        verbose=0
    ))
])


with mlflow.start_run(run_name="Production_RF_Fraud_Model"):
     mlflow.log_params({
        "pipeline_steps": [name for name, _ in fraud_pipeline.steps],
        "class_imbalance_ratio": class_weights[1],
        "feature_selection": "RandomForest_RFE",
        "n_estimators": 200,
        "max_depth": 15
    })
    
    print("Starting pipeline execution...")
    fraud_pipeline.fit(X_train, y_train)
    print("Training completed")
    
    fraud_probabilities = fraud_pipeline.predict_proba(X_val)[:, 1]
    fraud_predictions = fraud_pipeline.predict(X_val)
    
    pipeline_metrics = {
        "validation_auc": roc_auc_score(y_val, fraud_probabilities),
        "validation_ap": average_precision_score(y_val, fraud_probabilities),
        "validation_f1": f1_score(y_val, fraud_predictions),
        "fraud_precision": precision_score(y_val, fraud_predictions),
        "fraud_recall": recall_score(y_val, fraud_predictions)
    }
    
    mlflow.log_metrics(pipeline_metrics)
    mlflow.log_dict(
        classification_report(y_val, fraud_predictions, output_dict=True),
        "classification_report.json"
    )
    
    importance_plot = plt.figure(figsize=(12, 8))
    rf_model = fraud_pipeline.named_steps['fraud_classifier']
    feature_names = fraud_pipeline.named_steps['feature_selector'].kept_features_
    
    importances = rf_model.feature_importances_
    indices = np.argsort(importances)[::-1][:20]  # Top 20 features
    
    plt.title('Random Forest Feature Importance')
    plt.barh(range(len(indices)), importances[indices], align='center')
    plt.yticks(range(len(indices)), [feature_names[i] if i < len(feature_names) else f"feature_{i}" for i in indices])
    plt.xlabel('Relative Importance')
    plt.tight_layout()
    mlflow.log_figure(importance_plot, "feature_importance.png")
    plt.close()
    
    # Register model
    mlflow.sklearn.log_model(
        fraud_pipeline,
        "rf_fraud_detection_model",
        registered_model_name="Production_RF_Fraud_Pipeline"
    )
    
    print(f" RF Pipeline deployed | Best F1: {pipeline_metrics['validation_f1']:.4f}")

Starting pipeline execution...
Identified 12 columns to drop
Identified 238 highly correlated features to remove
Starting feature selection...
 Phase 2: Fine-tuning feature set...
Done! Selected 182 features.
Training completed


2025/04/29 10:57:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Production_RF_Fraud_Pipeline'.
2025/04/29 10:57:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Production_RF_Fraud_Pipeline, version 1
Created version '1' of model 'Production_RF_Fraud_Pipeline'.


 RF Pipeline deployed | Best F1: 0.4644
🏃 View run Production_RF_Fraud_Model at: https://dagshub.com/kechik21/ML_HW2.mlflow/#/experiments/3/runs/b53131f0aba94f1a94af2d41f3ead26a
🧪 View experiment at: https://dagshub.com/kechik21/ML_HW2.mlflow/#/experiments/3
